# Сборный проект

### Описание проекта
Вы работаете в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи вашего мобильного приложения.
Изучите воронку продаж. Узнайте, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?
После этого исследуйте результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясните, какой шрифт лучше.
<a id='table_of_contents'></a>

В данном проекте от меня, как от аналитика, требуется изучить и предобработать данные в логах, проанализировать адекватность данных в файле, изучить воронку продаж и результаты эксперимента. Сделать вывод об успешности эксперимента.

Мы располагаем следующими данными о логах:
* EventName — название события;
* DeviceIDHash — уникальный идентификатор пользователя;
* EventTimestamp — время события;
* ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

1. [Загрузка данных и подготовка к анализу](#step1) <br>
2. [Изучение и проверка данных](#step2) <br>
    2.1. [Удаление дубликатов и изменение названий](#step2_1) <br>
    2.2. [Добавление столбцов с датой и датой и временем](#step2_2) <br>
    2.3 [Количество событий в логе в разбивке по названию](#step2_3) <br>
    2.4 [Количество пользователей в логе](#step2_4) <br>
    2.5 [Среднее количество событий на пользователя](#step2_5) <br>
    2.6 [Максимальная и минимальная дата](#step2_6) <br>
    2.7 [Отбросим данные](#step2_7) <br>
3. [Изучение воронки событий](#step3) <br>
    3.1 [События в логах по частоте](#step3_1) <br>
    3.2 [События по числу пользователей](#step3_2) <br>
    3.3 [Построение цепочки событий](#step3_3) <br>
    3.4 [Построение воронки событий](#step3_4)     <br>
    3.5 [Построение воронки событий с учётом последовательности](#step3_5) <br>
4. [Анализ результатов эксперимента](#step4) <br>
    4.1 [Количество пользователей в каждой экс. группе](#step4_1) <br>
    4.2 [Проверка результатов А/А теста](#step4_2)     <br>
    4.3 [Проверка результатов A/B теста для каждой из контрольных групп](#step4_3) <br>
    4.4 [Проверка результатов A/B теста для объединённых контрольных групп](#step4_4) <br>
5. [Общий вывод](#step5)

<a id='step1'></a>
## Шаг 1. Откроем файл с данными и изучим общую информацию
[К оглавлению](#table_of_contents)

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import scipy.stats as stats
import plotly.graph_objects as go
warnings.filterwarnings("ignore")
import math as mth
from scipy import stats as st

In [ ]:
df.info()

### Таблица с данными содержит информацию о действиях пользователя в приложении, с указанием названия действия, временем, уникальным идентифиикатором пользователя, его совершившего, и контрольной группой,  которой этот пользователь принадлежал. Пропуск не обнаружено.

## Шаг 2. Подготовим данные
* Заменим названия столбцов на удобные для нас;
* Проверим пропуски и типы данных;
* Добавим столбец даты и времени, а также отдельный столбец дат; <br>
[К оглавлению](#table_of_contents)

<a id='step2_1'></a>
### Удалим дубликаты, исправим название столбцов, приведём к нижнему регистру строковый тип данных и понизим порядок числовых.

In [ ]:
df.drop_duplicates(inplace = True)

Появление дубликатов в логах, скорее всего, связано с техническими ошибками.

In [ ]:
df.rename(columns={'EventName': 'event_name', 'DeviceIDHash': 'user_id', 'EventTimestamp':'timestamp','ExpId':'exp_id'}, inplace=True)

In [ ]:
df['exp_id'] = df['exp_id'].astype('uint8')
df['timestamp'] = df['timestamp'].astype('uint32')
df['event_name'] = df['event_name'].str.lower()

<a id='step2_2'></a>
### Добавим столбцы даты и времени и столбец дат, проверим результаты предобработки:

In [ ]:
def convert_datetime(row):
    timestamp = row['timestamp'] 
    date = datetime.datetime.fromtimestamp(timestamp)
    return date
df['datetime'] = df.apply(convert_datetime, axis = 1)
df['date'] = df['datetime'].astype('datetime64[D]')

In [ ]:
df.info()

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Ошибки в таблицах исправлены. Данные подготовлены к дальнейшему анализу. 

</div>

<a id='step2'></a>
## Шаг 3. Изучим и проверим данные
* Сколько всего событий в логе?
* Сколько всего пользователей в логе?
* Сколько в среднем событий приходится на пользователя?
* Данными за какой период вы располагаете? Найдите максимальную и минимальную дату. Постройте гистограмму по дате и времени. * * * Можно ли быть уверенным, что у вас одинаково полные данные за весь период? Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Определим, с какого момента данные полные и отбросьте более старые. Данными за какой период времени мы располагаем на самом деле?
* Много ли событий и пользователей мы потеряли, отбросив старые данные?
* Проверим, что у вас есть пользователи из всех трёх экспериментальных групп. <br>

[К оглавлению](#table_of_contents)

<a id='step2_3'></a>
## Узнаем сколько событий в логе в разбивке по названию и всего:

In [ ]:
print(df['event_name'].value_counts(), '\n')
sum = df['event_name'].value_counts().sum()
print(f'Всего событий: {sum}')

<a id='step2_4'></a>
## Узнаем сколько всего пользователей в логе:

In [ ]:
df['user_id'].nunique()

<a id='step2_5'></a>
## Сколько в среднем событий приходится на пользователя?

In [ ]:
fig = plt.subplots(1, 3, figsize=(19.5, 6));
plt.subplot(131);
event_count = df.groupby('user_id')['event_name'].count()
event_count.plot(kind = 'hist', bins = 39, grid = True);
plt.title('Количество событий, совершаемых пользователем');

plt.subplot(132);
event_count = event_count[event_count < 100]
event_count.plot(kind = 'hist', bins = 39, grid = True);
plt.title('Количество событий, совершаемых пользователем (до 100)');

plt.subplot(133);
event_count = event_count[event_count < 40]
event_count.plot(kind = 'hist', bins = 39, grid = True);
plt.title('Количество событий, совершаемых пользователем (до 40)');

In [ ]:
event_count.median()

In [ ]:
event_count.mean()

При оценке среднего будем использовать именно моду, в силу ярко выраженной унимодальности графика, наибольшее число пользователей соответствуют именно этому значению. 
#### Среднее количество событий, приходящихся на одного пользователя - 5

## Анализ времени:
<a id='step2_6'></a>

### Посмотрим на максимальную и минимальную дату:

In [ ]:
df['date'].min()

In [ ]:
df['date'].max()

### Построим гистограмму:

In [ ]:
df['date'].hist(bins = 14);
plt.xticks(rotation='vertical');
plt.title('Распределение количества событий по датам');

#### Можно заметить, что с началом нового месяца количество событий сильно растёт с 2-х тысяч до 36, изучим вопрос поподробнее и отсечём все данные с прошлого месяца:

In [ ]:
top_date = df['date'].value_counts().reset_index().head(7)
top_date.columns = ['date','count']
top_date

<a id='step2_7'></a>
#### Создадим таблицу new_df только из новых данных, из старых данных создадим таблицу discarded: 

In [ ]:
new_df = df.merge(top_date, how = 'right', on = 'date').reset_index(drop = True)

In [ ]:
discarded = df.merge(top_date, how = 'left', on = 'date').reset_index(drop = True)
discarded = discarded[discarded['count'].isna()]

#### Всего было отброшено 2828 событий, проверим, сколько  мы потеряли пользователей, отбросив старые данные:

In [ ]:
discarded['user_id'].nunique()

Мы потеряли 1451 пользователя, посчитаем процентные доли:

In [ ]:
len(discarded) / len(df) * 100

In [ ]:
discarded['user_id'].nunique() / df['user_id'].nunique() * 100

### Отбросив старые данные (1.16% всего датафрейма), мы потеряли 19.2% всех уникальных пользователей.

## Проверьте, что у вас есть пользователи из всех трёх экспериментальных групп.

In [ ]:
new_df['exp_id'].value_counts()

<a id='step3'></a>
## Шаг 4. Изучим воронку событий
* Посмотрим, какие события есть в логах, как часто они встречаются. Отсортируем события по частоте.
* Посчитаем, сколько пользователей совершали каждое из этих событий. Отсортируем события по числу пользователей. Посчитаем долю пользователей, которые хоть раз совершали событие.
* Предположим, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не будем учитывать при расчёте воронки.
* По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C посчитаем отношение числа пользователей с событием B к количеству пользователей с событием A, а также отношение числа пользователей с событием C к количеству пользователей с событием B.
* На каком шаге теряем больше всего пользователей?
* Какая доля пользователей доходит от первого события до оплаты? <br>

[К оглавлению](#table_of_contents)

<a id='step3_1'></a>
### Посмотрите, какие события есть в логах, как часто они встречаются. Отсортируйте события по частоте.

In [ ]:
event_counts = new_df['event_name'].value_counts().reset_index()
event_counts.columns  = ['event_name','count']
event_counts

In [ ]:
ax = go.FigureWidget()
ax.add_bar(x= event_counts['event_name'], y = event_counts['count'],
           textposition='auto', text= event_counts['count'])
ax.layout.title = 'Как часто встречаются события в логах?'
ax.update_layout(yaxis_title = "Количество событий", xaxis_title = "Название события",  width=800, height = 580)
ax.show()

<a id='step3_2'></a>
### Посчитайте, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитайте долю пользователей, которые хоть раз совершали событие.

In [ ]:
events_by_users = new_df.pivot_table(values = 'user_id', 
                                     columns = 'event_name',  
                                     aggfunc = {'user_id':'nunique'}).T.reset_index().sort_values(by = 'user_id', 
                                                                                                  ascending = False).reset_index(drop = True)

events_by_users['total_share'] = round(events_by_users['user_id'] / len(new_df['user_id'].unique()),2)
events_by_users

Даже на первом шаге воронки мы не имеем 1, это говорит о том, что проход через все шаги воронки не обязателен, некоторые пользователи попадая в приложение не сразу идут на главное меню, возможно сначала смотрят **tutorial**, либо приходят по ссылке уже сразу на какой-то товар.

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Даже на первом шаге воронки мы не имеем 1. О чем это может нам говорить? Обязателен ли проход через все шаги воронки? 
    
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Да, сайты устроены довольно сложно. Поэтому прямой проход по воронке и необязателен. 

</div>

In [ ]:
ax = go.FigureWidget()
ax.add_bar(x= events_by_users['event_name'], y = events_by_users['user_id'],
           textposition='auto', text= events_by_users['user_id'])
ax.layout.title = 'События по числу пользователей'
ax.update_layout(yaxis_title = "Количество пользователей", xaxis_title = "Название события",  width=800, height = 580)          
ax.show()

<a id='step3_3'></a>
### Предположим, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не будем учитывать при расчёте воронки.

Перевод:
- **mainscreenappear** - появится главный экран;
- **offersscreenappear** - появится экран предложений;
- **cartscreenappear** - появилась корзина;
- **paymentscreensuccessful** - экран платёжного успешен;
- **tutorial** - руководство.

В последовательную цепочку выстраиваются - **mainscreenappear** - **offersscreenappear** - **cartscreenappear** - **paymentscreensuccessful**

In [ ]:
new_df = new_df[new_df['event_name'] != 'tutorial'].reset_index(drop = True)

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Последовательность событий определена. Соглашусь, tutorial не относится к ней. Этот шаг является обучением и не относится к последовательности событий.  
    
Также можно добавить и группировку по номеру эксперимента (246, 247, 248) в столбцах. Так мы посмотрим на то, согласуются ли данные разных групп между собой.  
    
</div>

<a id='step3_4'></a>
### По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C посчитаем отношение числа пользователей с событием B к количеству пользователей с событием A, а также отношение числа пользователей с событием C к количеству пользователей с событием B. На каком шаге теряется больше всего пользователей?

In [ ]:
# Сделаем новую таблицу, исключив повторяющиеся события для одного и того же пользователя
unique_action = new_df.drop_duplicates(subset = ['event_name', 'user_id']).reset_index(drop = True)

def definition_id(row):
    event_name = row['event_name'] 
    
    if event_name == 'mainscreenappear':
        result = 0
    elif event_name == 'offersscreenappear':
        result = 1
    elif event_name == 'cartscreenappear':
        result = 2
    elif event_name == 'paymentscreensuccessful':
        result = 3
    return result

unique_action['count'] = unique_action.apply(definition_id, axis = 1)
unique_action.rename(columns=lambda x: x.replace('count', 'evna_id'), inplace=True)

In [ ]:
percent = []
percent.append(round(len(unique_action[unique_action['evna_id'] == 0]) / len(unique_action[unique_action['evna_id'] == 0]),3)*100)
percent.append(round(len(unique_action[unique_action['evna_id'] == 1]) / len(unique_action[unique_action['evna_id'] == 0]),3)*100)
percent.append(round(len(unique_action[unique_action['evna_id'] == 2]) / len(unique_action[unique_action['evna_id'] == 1]),3)*100)
percent.append(round(len(unique_action[unique_action['evna_id'] == 3]) / len(unique_action[unique_action['evna_id'] == 2]),3)*100)

In [ ]:
lens = []
lens.append(len(unique_action[unique_action['evna_id'] == 0]))
lens.append(len(unique_action[unique_action['evna_id'] == 1]))
lens.append(len(unique_action[unique_action['evna_id'] == 2]))
lens.append(len(unique_action[unique_action['evna_id'] == 3]))

In [ ]:
d = {'total': lens, 'percent' : percent, 'relationship': ['mainscreenappear',
    'offersscreenappear',
    'cartscreenappear', 'paymentscreensuccessful']}
percent_data = pd.DataFrame(data = d)
percent_data

In [ ]:
fig2 = go.Figure(go.Funnel(
    y = percent_data['relationship'],
    x = percent_data['total'],
    textposition = "inside",
    textinfo = "value+percent initial"))
fig2.update_layout(title='Воронка событий')

Больше всего пользователей теряется после первого шага, отсекается более трети всех пользователей. До последнеого этапа доходит чуть меньше половины пользователей.

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Крутая визуализация. Воронку мы изучили. Около половины клиентов доходит с первого этапа до последнего.
    
</div>

<a id='step3_5'></a>
### Какая доля пользователей доходит от первого события до оплаты?

In [ ]:
unique_action_pivot = unique_action.pivot_table(columns = 'event_name', values = 'datetime', aggfunc = 'min', index = 'user_id')

In [ ]:
def asd (row):
    mainscreenappear = row['mainscreenappear']
    offersscreenappear = row['offersscreenappear']
    paymentscreensuccessful = row['paymentscreensuccessful']
    cartscreenappear = row['cartscreenappear']
    if (mainscreenappear == mainscreenappear): 
        result = 1
        if (mainscreenappear <= offersscreenappear):
            result = 2
            if (offersscreenappear <= cartscreenappear):
                result = 3
                if (cartscreenappear <= paymentscreensuccessful):
                    result = 4
    else: result = 0
    return result

In [ ]:
unique_action_pivot['successful'] = unique_action_pivot.apply(asd, axis = 1)

In [ ]:
unique_action_pivot = unique_action_pivot['successful'].value_counts().reset_index()

In [ ]:
unique_action_pivot = unique_action_pivot[unique_action_pivot['index'] > 0]
unique_action_pivot['index'] = unique_action_pivot['index'].astype('object')
unique_action_pivot['index'] = ['mainscreenappear','offersscreenappear','cartscreenappear','paymentscreensuccessful']
unique_action_pivot 

In [ ]:
fig2 = go.Figure(go.Funnel(
    y = unique_action_pivot['index'],
    x = unique_action_pivot['successful'],
    textposition = "inside",
    textinfo = "value+percent initial"))
fig2.update_layout(title='Воронка событий с учётом последовательности')

#### 14% пользователей проходят по каждому шагу воронки

<a id='step4'></a>
## Шаг 5. Изучим результаты эксперимента
* Сколько пользователей в каждой экспериментальной группе?
* Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверим, находят ли статистические критерии разницу между выборками 246 и 247.
* Выберим самое популярное событие. Посчитаем число пользователей, совершивших это событие в каждой из контрольных групп. Посчитаем долю пользователей, совершивших это событие. Проверим, будет ли отличие между группами статистически достоверным. Проделаем то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?
* Аналогично поступим с группой с изменённым шрифтом. Сравним результаты с каждой из контрольных групп в отдельности по каждому событию. Сравним результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?
* Какой уровень значимости стоит выбрать при проверке статистических гипотез выше? Посчитаем, сколько проверок статистических гипотез мы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если мы хотим изменить его, проделайте предыдущие пункты и проверьте свои выводы. <br>

[К оглавлению](#table_of_contents)

<a id='step4_1'></a>
### Сколько пользователей в каждой экспериментальной группе?

In [ ]:
exp_id_users = new_df.groupby('exp_id')['user_id'].nunique().reset_index()
exp_id_users

In [ ]:
print(100 - 2483 / 2535 * 100)
print(100 - 2512 / 2535 * 100)

<a id='step4_2'></a>
### Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.

In [ ]:
sample_246 = new_df[new_df['exp_id'] == 246].reset_index(drop = True)
sample_247 = new_df[new_df['exp_id'] == 247].reset_index(drop = True)
sample_248 = new_df[new_df['exp_id'] == 248].reset_index(drop = True)

#### Ответ на этот вопрос в пункте ниже...

### Выберим самое популярное событие. Посчитаем число пользователей, совершивших это событие в каждой из контрольных групп. Посчитаем долю пользователей, совершивших это событие. Проверим, будет ли отличие между группами статистически достоверным. Проделаем то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?

In [ ]:
users_event_246 = sample_246.groupby('event_name')['user_id'].nunique().reset_index()
all_users = len(sample_246['user_id'].unique())
users_event_246['share'] = users_event_246['user_id'] / all_users
users_event_246

In [ ]:
users_event_247 = sample_247.groupby('event_name')['user_id'].nunique().reset_index()
all_users = len(sample_247['user_id'].unique())
users_event_247['share'] = users_event_247['user_id'] / all_users
users_event_247

In [ ]:
users_event_248 = sample_248.groupby('event_name')['user_id'].nunique().reset_index()
all_users = len(sample_248['user_id'].unique())
users_event_248['share'] = users_event_248['user_id'] / all_users
users_event_248

<a id='step4_3'></a>
## Проведём z-тест

In [ ]:
alpha = .1 # критический уровень статистической значимости

* $H_0$  в обеих генеральных совокупностях доли равны.
* $H_1$  в обеих генеральных совокупностях доли различаются статистически значимо.
###  Эти гипотезы будут проверяться во всех 16-и тестах снизу:

In [ ]:
def z_test(sample_first,sample_second,event_name:object, exp_id1 = None, exp_id2 = None):
    if exp_id1 == None:
        in_group = np.array([sample_first[sample_first['event_name'] == event_name]['user_id'],
                             sample_second[sample_second['event_name'] == event_name]['user_id']])
        users = np.array([exp_id_users[exp_id_users['exp_id'] != 248]['user_id'].sum(),
                          exp_id_users[exp_id_users['exp_id'] == 248]['user_id']])   
    else:
        in_group = np.array([sample_first[sample_first['event_name'] == event_name]['user_id'],
                                sample_second[sample_second['event_name'] == event_name]['user_id']])
        users = np.array([exp_id_users[exp_id_users['exp_id'] == exp_id1]['user_id'],
                          exp_id_users[exp_id_users['exp_id'] == exp_id2]['user_id']])
    
    
    p1 = in_group[0]/users[0]
    p2 = in_group[1]/users[1]
    p_combined = (in_group[0] + in_group[1]) / (users[0] + users[1])
    difference = p1 - p2 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/users[0] + 1/users[1])) 
    distr = st.norm(0, 1)  

    p_value = (1 - distr.cdf(abs(z_value))) * 2 
    print(f'Проведён z-тест для  выборок по event_name = {event_name}')
    print('p-значение: ', p_value)

    if p_value < alpha: 
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

In [ ]:
z_test(users_event_246,users_event_247,'mainscreenappear', 246, 247)

In [ ]:
z_test(users_event_246,users_event_247,'offersscreenappear', 246, 247)

In [ ]:
z_test(users_event_246,users_event_247,'cartscreenappear', 246, 247)

In [ ]:
z_test(users_event_246,users_event_247,'paymentscreensuccessful', 246, 247)

### Разбиение на группы работает корректно! Статистически значимых различий между долями для двух АА групп не найдено.

In [ ]:
def all_ztests(sample_1, sample_2, exp_id1 = None,exp_id2 = None):
    if exp_id1 == None:
        z_test(sample_1,sample_2,'mainscreenappear')
        print()
        z_test(sample_1,sample_2,'offersscreenappear')
        print()
        z_test(sample_1,sample_2,'cartscreenappear')
        print()
        z_test(sample_1,sample_2,'paymentscreensuccessful')
    else:
        z_test(sample_1,sample_2,'mainscreenappear', exp_id1, exp_id2)
        print()
        z_test(sample_1,sample_2,'offersscreenappear', exp_id1, exp_id2)
        print()
        z_test(sample_1,sample_2,'cartscreenappear', exp_id1, exp_id2)
        print()
        z_test(sample_1,sample_2,'paymentscreensuccessful', exp_id1, exp_id2)

### Сравним 246 и 248  группы:

In [ ]:
all_ztests(users_event_246, users_event_248, 246, 248)

### Сравним 247 и 248  группы:

In [ ]:
all_ztests(users_event_247, users_event_248, 247, 248)

<a id='step4_4'></a>
### Сравним объединённую 246 и 247 с 248 экспериментальной группой:

In [ ]:
sample246_247 = new_df[new_df['exp_id'] != 248].reset_index(drop = True)
users_event_246_247 = sample246_247.groupby('event_name')['user_id'].nunique().reset_index()

all_users = len(sample246_247['user_id'].unique())

users_event_246_247['share'] = users_event_246_247['user_id'] / all_users
users_event_246_247

In [ ]:
z_test(users_event_246_247,users_event_248,'mainscreenappear')

In [ ]:
all_ztests(users_event_246_247, users_event_248)

### Для уровня значимости alpha = 0.05 - 246,247 и 248 группы не имеют статистических различий.
### Для уровня значимости alpha = 0.1 между 246 и 248 группы появляется статистическая значимость.

<a id='step5'></a>
# Общий вывод:

[К оглавлению](#table_of_contents)
#### Мы получили данные
о логах мобильного приложения, после предобработки (удаления дубликатов, понижения порядка числовых атрибутов и приведения в нижний регистр строковы) 
#### приступили к анализу:

* Узнали, что всего в эксперименте участвовал 7551 пользователь;
* Узнали, что в среднем пользователь совершает 5 действий;
* Выяснили, что не весь период соответствует времени проведения эксперимента, отбросили ненужное время (всё до 1-го августа) и посмотрели, как много пользователей мы потеряли, оказалось, что отбросив 1.16% датафрейма, мы потеряли 19.2% всех уникальных пользователей. 
#### Изучили воронку продаж:

выяснили, что записей с удачным совершением покупки примерно в 2 раза меньше, чем записей с просмотром основного меню. <br>
При движении по воронке продаж: **mainscreenappear - offersscreenappear - cartscreenappear - paymentscreensuccessful** конверсия от первого шага до последнего - 14%. 
#### Изучили результаты эксперимента:
* Для уровня значимости alpha = 0.05 - 246,247 и 248 группы не имеют статистических различий.
* Для уровня значимости alpha = 0.1 между 246 и 248 группы появляется статистическая значимость.
Доля событий статистически значимо не изменилась, для уровня значимости 0.05%, это говорит о том, что изменение шрифта во всём приложении не улучшило и не ухудшило ситуацию по показателям, поскольку изначально и не было задачи повысить конверсию в результате эксперимента, а была задача не распугать пользователей, то можно считать, что **эксперимент прошел удачно.**